# Análisis de Regresión para la Predicción del Valor de Mercado en el Fútbol
## Práctica personal - Carlos Pujols


Procedencia del dataset: https://www.kaggle.com/datasets/davidcariboo/player-scores?select=player_valuations.csv 

### Importación de dataset y limpieza de datos


In [40]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el primer archivo CSV
df = pd.read_csv('players.csv')

# Seleccionamos las columnas relevantes
df1 = df[[
    'player_id',
    'market_value_in_eur',           # Valor de mercado en euros
    'highest_market_value_in_eur',   # Máximo valor de mercado en euros
    'position',                      # Posición del jugador
    'date_of_birth',                 # Fecha de nacimiento
    'current_club_name',             # Nombre del club actual
    'country_of_birth',              # País de nacimiento
    'country_of_citizenship',        # País de ciudadanía
    'height_in_cm',                  # Altura en centímetros
    'foot',                          # Pie dominante
    'sub_position',                  # Subposición (si aplica)
    'agent_name'                     # Nombre del agente
]]

df2 = pd.read_csv('appearances.csv')

df2f = df2[[
    'player_id',       # Identificador del jugador
    'goals',           # Goles anotados
    'assists',         # Asistencias
    'minutes_played',  # Minutos jugados
    'yellow_cards',    # Tarjetas amarillas
    'red_cards'        # Tarjetas rojas
]]

# Unimos los dos datasets usando 'player_id' como clave
dataMergerado = pd.merge(df1, df2, on='player_id', how='left')

# Imputación con la media de las columnas
dataMergerado.dropna(how='any', inplace=True)

# Convertir las variables categóricas 'position' y 'foot' a variables dummy (One-Hot Encoding)
dataMergerado = pd.get_dummies(dataMergerado, columns=['position', 'foot'], drop_first=True)

# Crear dos instancias de StandardScaler
scaler_X = StandardScaler()  # Para las variables predictoras
scaler_y = StandardScaler()  # Para la variable objetivo

# Normalizar las variables predictoras
dataMergerado['minutes_played'] = scaler_X.fit_transform(dataMergerado[['minutes_played']]).flatten()

X = dataMergerado[['goals', 'assists', 'minutes_played', 'yellow_cards', 'red_cards'] + [col for col in dataMergerado.columns if 'position_' in col]]  # Variables predictoras
y = dataMergerado['market_value_in_eur']  # Variable objetivo (valor de mercado)

# Normalizar la variable objetivo
y = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()  # Ajustamos la forma para que funcione con fit_transform

# Normalizar las variables predictoras
X_normalized = scaler_X.fit_transform(X)  # Normalización de X

# Mostrar las primeras filas de X_normalized y y para verificar
print(X_normalized[:5])  # Primeras 5 filas de X normalizadas
print(y[:5])  # Primeros 5 valores de y normalizados


[[ 2.6834058  -0.26946064  0.46389541  2.34261996 -0.06066782 -0.71576251
  -0.27228617 -0.66637056]
 [-0.29471166 -0.26946064  0.69797598  2.34261996 -0.06066782 -0.71576251
  -0.27228617 -0.66637056]
 [ 5.66152326 -0.26946064  0.6645359   2.34261996 -0.06066782 -0.71576251
  -0.27228617 -0.66637056]
 [ 2.6834058  -0.26946064  0.69797598 -0.40219048 -0.06066782 -0.71576251
  -0.27228617 -0.66637056]
 [-0.29471166 -0.26946064  0.69797598 -0.40219048 -0.06066782 -0.71576251
  -0.27228617 -0.66637056]]
[-0.35993308 -0.35993308 -0.35993308 -0.35993308 -0.35993308]


## Regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print(f"Error cuadrático: {mse:.2f}")

# Coeficientes del modelo
print("Coeficientes del modelo:")
print(model.coef_)

Error cuadrático: 0.98
Coeficientes del modelo:
[ 0.04474027  0.03020574  0.03674768 -0.01693432 -0.00308647 -0.04272704
 -0.03457826  0.00370767]


# Visualizacion de coeficientes de posiciones

In [42]:
# Obtener las columnas que corresponden a las posiciones
position_columns = [col for col in dataMergerado.columns if 'position_' in col]

# Obtener los coeficientes correspondientes a las posiciones
position_coefficients = model.coef_[-len(position_columns):]

# Crear un DataFrame con las posiciones y sus coeficientes
position_df = pd.DataFrame({
    'position': position_columns,
    'coefficient': position_coefficients
})

# Ordenar por el coeficiente para ver las posiciones con peores valores (coeficientes negativos)
position_df_sorted = position_df.sort_values(by='coefficient', ascending=True)

# Mostrar las posiciones con los peores valores de mercado
print(position_df_sorted)


              position  coefficient
0    position_Defender    -0.042727
1  position_Goalkeeper    -0.034578
2    position_Midfield     0.003708


# Conclusiones sobre el resultado
A continuación se presentan algunas conclusiones clave basadas en los resultados obtenidos:

- **Impacto de las estadísticas de rendimiento**: El valor de mercado de un jugador está relacionado con su rendimiento en el campo. En particular, un mayor número de goles, asistencias y minutos jugados se asocia con un incremento en el valor de mercado del jugador. Esto nos sugiere que los jugadores más activos y productivos tienen una mayor demanda en el mercado.

- **Efecto de las sanciones**: En cambio, la acumulación de tarjetas amarillas y rojas tiene un efecto negativo sobre el valor de mercado del jugador. Cuantas más tarjetas reciba un jugador, su valor disminuye, lo que podría reflejar la percepción de un jugador menos disciplinado o confiable.

- **Influencias de la posición en el campo**: Las posiciones también tienen un papel importante en la determinación del valor de mercado. Los jugadores en posiciones más ofensivas, como pueden ser los mediocampistas, tienden a tener un valor de mercado más alto, dado que tienen más oportunidades para anotar goles y asistir. Por el contrario, los defensores y porteros, que suelen estar más cerca de su propia portería, tienen un valor de mercado inferior, ya que sus contribuciones a los goles son más limitadas.

En resumen, el valor de mercado de un jugador está influenciado por su rendimiento individual en el campo y por su posición en el mismo, con los jugadores más involucrados en la ofensiva y menos propensos a recibir sanciones generalmente teniendo de un valor de mercado más alto.